In [11]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

path = 'D:/pesquisas_insper'
url = f'https://www.constituteproject.org/constitutions?lang=en&status=in_force&status=is_draft'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
items = soup.find_all('a')
lista_links = list()
for item in items:
    link = item.get('href')
    try:
        if link.startswith('/constitution/'):
            lista_links.append(link)
    except:
        pass
lista_links = lista_links[1:]
country = list()
year = list()
final_links = list()
for link in lista_links:
    country.append(link[14:-13])
    year.append(link[-12:-8])
    final_links.append('https://www.constituteproject.org'+link)
df = pd.DataFrame()
df['country'] = country
df['year'] = year
df['link'] = final_links

In [12]:
contitution = list()
for url in final_links:
    try:
        text = ''
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        items = soup.find_all('section',{'id':'s0'})[0]
        for item in items:
            text = text + ' ' + item.text
        contitution.append(text)
    except:
        contitution.append(None)
df['constitution'] = contitution
df


,country,year,link,constitution
0,Afghanistan,2004,https://www.constituteproject.org/constitution...,"\n \n\nSource of constitutional authority, P..."
1,Albania,2016,https://www.constituteproject.org/constitution...,"\n \n\nPreamble, Motives for writing constit..."
2,Algeria,2020,https://www.constituteproject.org/constitution...,"\n \n\nReference to country's history, Motiv..."
3,Andorra,1993,https://www.constituteproject.org/constitution...,"\n \n\nPreamble, Motives for writing constit..."
4,Angola,2010,https://www.constituteproject.org/constitution...,"\n \n\nPreamble, Motives for writing constit..."
...,...,...,...,...
201,Vanuatu,2013,https://www.constituteproject.org/constitution...,"\n \n\nMotives for writing constitution, Ref..."
202,Venezuela,2009,https://www.constituteproject.org/constitution...,"\n \n\nReference to fraternity/solidarity, R..."
203,Yemen_,015D,https://www.constituteproject.org/constitution...,\n \nChapter I General Foundations\n\nSectio...
204,Zambia,2016,https://www.constituteproject.org/constitution...,"\n \n\nMotives for writing constitution, Sou..."


In [13]:
import re

preamble = list()
others = list()
for text in df.constitution:
    try:
        stop = re.search(f'chapter\s[i1]|chapter\sone|part\sone|part\s[i1]|section\s[i1]|section\sone|article\s[i1]|article\sone|i\.\s?preliminary',text,re.I|re.S).span()[0]
        preamble.append(text[:stop])
        others.append(text[stop:])
    except:
        preamble.append(None)
        others.append(None)
df['preamble'] = preamble
df['others'] = others

len1 = list()
len2 = list()
for _,row in df.iterrows():
    try:
        len1.append(len(row.preamble.split()))
    except:
        len1.append(None)
    
    try:
        len2.append(len(row.others.split()))
    except:
        len2.append(None)

df['length_preamble'] = len1
df['length_const'] = len2

In [14]:
from sentence_transformers import SentenceTransformer, util
import os
import pandas as pd

path = 'D:/pesquisas_insper/pesquisas_insper'
# df = pd.read_csv(os.path.join(path, 'dados_constituicao.csv'))
model = SentenceTransformer('all-MiniLM-L6-v2')
df.head()

,country,year,link,constitution,preamble,others,length_preamble,length_const
0,Afghanistan,2004,https://www.constituteproject.org/constitution...,"\n \n\nSource of constitutional authority, P...","\n \n\nSource of constitutional authority, P...",Chapter I. State\n\n\nType of government envis...,308.0,10730.0
1,Albania,2016,https://www.constituteproject.org/constitution...,"\n \n\nPreamble, Motives for writing constit...","\n \n\nPreamble, Motives for writing constit...",PART ONE. BASIC PRINCIPLES\n\nArticle 1\n\n\nT...,133.0,23427.0
2,Algeria,2020,https://www.constituteproject.org/constitution...,"\n \n\nReference to country's history, Motiv...","\n \n\nReference to country's history, Motiv...",CHAPTER I. ALGERIA\n\n\nType of government env...,1377.0,16348.0
3,Andorra,1993,https://www.constituteproject.org/constitution...,"\n \n\nPreamble, Motives for writing constit...","\n \n\nPreamble, Motives for writing constit...",Article 1\nAndorra is a Democratic and Social ...,268.0,8835.0
4,Angola,2010,https://www.constituteproject.org/constitution...,"\n \n\nPreamble, Motives for writing constit...","\n \n\nPreamble, Motives for writing constit...","Article 158 of the Constitutional Law of 1992,...",117.0,27050.0


In [15]:
cosine_scores = list()
for _, row in df.iterrows():
    try:
        sentences = [row.preamble,
                    row.others]

        #Compute embeddings
        embeddings = model.encode(sentences, convert_to_tensor=True)
        cosine_scores.append(util.cos_sim(embeddings, embeddings)[0][1])
    except:
        cosine_scores.append(None)
df['score'] = cosine_scores
df.to_excel(os.path.join(path,'final_test.xlsx'),index = False)